# R1.04 Méthodes d'optimisation

# TP2 A


_Thibault Godin & Lucie Naert_

_IUT de Vannes, BUT Informatique_



# $\mathbf{SAT}$ solver

In [ ]:
import numpy as np 
import pycosat
import itertools
import sys, getopt 
import time
from pprint import pprint

ModuleNotFoundError: No module named 'pycosat'

*********

# $\mathbf{ SAT}$


En première année, vous avez vu les concepts d'assertions et de formules (booléennes) logiques. Par exemple la formule

$$A :=  (\neg P \wedge Q) \vee  (P\wedge \neg Q) $$


qui a pour table de vérité <a name="cite_ref-1"></a>[<sup>[1]</sup>](#cite_note-1) :


|P|Q|A|
| ------------- |:-------------:| -----:|
|V|V|V|
|V|F|F|
|F|V|F|
|F|F|V|


Étant donnée une assertion $A$, on dira que l'assertion est _satisfaisable_ (satisfiable) s'il existe des valeurs de vérités pour les variables atomiques (ici $P$ et $Q$) telle que $A$ soit vraie. Dans notre exemple $A$ est satisfaisable, et un _témoin_ (witness) est le choix de valeurs $P=F$ et $Q=F$. 


La question _"étant donnée une assertion logique $A$, déterminer si elle est satisfaisable"_ est un problème majeur en informatique, appelé [SAT](https://fr.wikipedia.org/wiki/Probl%C3%A8me_SAT)


*********


**_question 1_**
Déterminer si les assertions suivantes sont satisfaisables, et donner un témoin si elle le sont.


- $A_1 :=  \neg (P \wedge Q) \vee R $
- $A_2 :=  \neg (P \wedge Q) \vee P $
- $A_3 :=  \neg (P \wedge Q \wedge \neg R) \vee (P \wedge R)$
- $A_4 :=  P \Longleftrightarrow Q \vee \neg(P \wedge Q) $



<a name="cite_note-1"></a>1. [^](#cite_ref-1) On note généralement cet opérateur $P \Longleftrightarrow Q$



- $A_1 : P = false, Q = false, R = true$
- $A_2 : P = false, Q = true$
- $A_3 : P = false, Q = false, R = true$
- $A_4 : P = false, Q = false$

*********

## CNF

Pour les problèmes de satisfiabilité, on a généralement un grand nombre de variables booléennes. On les notera généralement $x_1, x_2, \dots, x_n$, et on dira que $n$ est la _taille_ de l'instance.
De plus, on a vu en R1.06 que toute formule peut s'écrire à l'aide des seules opérations $\neg, \vee, \wedge$, et l'on peut même exiger que les $\neg$ ne porte que sur des propositions atomiques (les $x_i$).

*********
**_question 2_**

Transformer les propositions suivantes de manière à n'utiliser que les opérations $\neg, \vee, \wedge$, et que les $\neg$ ne porte que sur des propositions atomiques.
- $A_1 :=  \neg (P \wedge Q) \vee P $
- $A_1 :=  \neg (P \wedge Q \wedge \neg R) \vee (P \wedge R)$
- $A_3 := \neg (P \Longleftarrow Q)$
- $A_4 :=  P \Longleftrightarrow Q \vee \neg(P \wedge Q) $


Pour faciliter la saisie des assertion, on va normaliser un peu plus encore<a name="cite_ref-3"></a>[<sup>[3]</sup>](#cite_note-3) : on va demander à ce que nos formules soient des conjonctions (des "et") de disjonctions (des "ou") de variables. On appelle cette forme [CNF](https://fr.wikipedia.org/wiki/Forme_normale_conjonctive).

Par exemple $(x_1 \vee x_2 \vee \neg x_3) \wedge (x_1 \vee x_3)$ et  $(x_1 \vee \neg x_5 \vee x_4) \wedge (\neg x_1 \vee  x_5 \vee x_3 \vee x_4) \wedge (\neg x_3 \vee \neg x_4)$ est en CNF,
tandis que $ \neg(x_1 \wedge x_2 \wedge \neg x_3) \vee (x_1 \wedge x_3)$ et $(x_1 \wedge \neg x_5 \wedge x_4) \vee  (\neg x_1 \wedge  x_5 \wedge x_3 \wedge x_4) \vee  (\neg x_3 \wedge \neg x_4)$ (le dernier exemple est en forme normale disjonctive, DNF)


On appelera _clause_ les parties entre deux $\wedge$'s consécutifs.

<a name="cite_note-3"></a>2. [^](#cite_ref-3) On peut aller encore plus loin en imposant que la formule soit une disjonction (un "ou") de conjonctions (des "et") d'au plus $3$ variables. On optient le problème [3-SAT](https://fr.wikipedia.org/wiki/Probl%C3%A8me_3-SAT) qui est équivalent au problème $\mathbf{SAT}$


- $A_1 := (\neg P \vee \neg Q) \vee P$
- $A_2 := (\neg P \vee \neg Q \vee R) \vee (P \wedge R)$
- $A_3 := \neg P \wedge Q$
- $A_4 := P \Longleftrightarrow Q \vee (\neg P \vee \neg Q)$

*********
**_question 3_**

Mettre les deux derniers exemples en CNF.

$A_1 := (\neg P \vee \neg Q) \wedge (P \vee \neg Q)$

$A_2 := (\neg P \vee \neg Q \vee R) \wedge (P \vee R) \wedge (\neg P \vee \neg Q \vee R) \wedge (\neg P \vee R)$

$A_3 := (\neg P \vee Q) \wedge (\neg Q \vee Q)$

$A_4 := (P \vee Q \vee \neg P) \wedge (P \vee Q \vee \neg Q) \wedge (\neg P \vee P \vee \neg Q) \wedge (\neg P \vee \neg Q \vee Q)$

*********
**_question 4_**


Essayer de résoudre quelques instances en CNF sur [SAT Game](http://www.cril.univ-artois.fr/~roussel/satgame/satgame.php?level=2&lang=fr)


*********

In [ ]:
cnf = [[1, -5, 4], [-1, 5, 3, 4], [-3, -4]]

représente la formule 


 $(x_1 \vee \neg x_5 \vee x_4) \wedge (\neg x_1 \vee  x_5 \vee x_3 \vee x_4) \wedge (\neg x_3 \vee \neg x_4)$ 
 
 
Ici, on a 5 variables et 3 clauses, la première clause étant $(x_1 \vee \neg x_5 \vee x_4)$. 

*********


On définit une fonction retournant le nombre de variables et de clauses d'une formule en CNF

In [ ]:
def cnflen(cnf):
    #returns the number of variables and of clauses of a boolean formula given under CNF
    return max([abs(max(max(cnf, key=max))),abs(min(min(cnf, key=min)))]),len(cnf)

cnflen(cnf)



(5, 3)

*********

# $\mathbf{SAT}$-Solver



La résolution algorithmique d'un problème $\mathbf{SAT}$ est assez facile : on peut tester tous les cas possibles, jusqu'à ce que l'on trouve un témoin ou que l'on ait essayer toutes les combinaisons. Pour une instance de taille $n$ il "suffit" donc de tester $2^n$ cas.


*********

**_question 5_**


Écrire une fonction qui prend en entrée une formule booléenne en CNF à  $n$ variables et un tableau de valeurs booléennes et retourne l'évaluation de la formule sur ces valeurs.

In [ ]:
def Eval(cnf, wit):
    # Obtention du nombre de variables dans la formule CNF
    n, c = cnflen(cnf)
    
    # Évaluation de chaque clause
    for clause in cnf:
        clause_val = False
        for literal in clause:
            var = abs(literal)
            val = wit[var - 1] if literal > 0 else not wit[var - 1]
            clause_val = clause_val or val
        if not clause_val:
            # Si une clause est fausse, la formule est fausse
            return False
    
    # Si toutes les clauses sont vraies, la formule est vraie
    return True


#True             
wit=[1, 0, 0, 0, 1]               
cnf = [[1, -5, 4], [-1, 5, 3, 4], [-3, -4]]
Eval(cnf,wit)


#False
wit=[0, 0, 1, 1, 0]               
cnf = [[1, -5, 4], [-1, 5, 3, 4], [-3, -4]]
Eval(cnf,wit)


print(cnflen([[1],[2]]))
print(Eval([[1],[2]],[1,0]))


(2, 2)
True


*********

**_question 6_**


Écrire une fonction qui essaie de resoudre une formule en CNF par force-brute (en essayant toutes les possibilités).
_on pourra utiliser le module itertool.product de python_


In [ ]:
def bruteSAT(cnf):
    n, c = cnflen(cnf)
    for wit in itertools.product([False, True], repeat=n):
        if Eval(cnf, wit):
            return wit
    return None


print(bruteSAT(cnf))
print(bruteSAT([[1,2]]))
print(bruteSAT([[1],[2]]))
print(bruteSAT([[1,2],[-1,2],[1,-2]]))
print(bruteSAT([[1,2],[-1,2],[1,-2],[-1,-2]]))



-1
-1
-1
-1
-1


*********

## PycoSAT

On ne sait pas aujourd'hui s'il existe un algorithme polynômial (_ie_ s'executant en temps raisonnable) pour résoudre $\mathbf{SAT}$. Cependant ce problème, bien qu'abstrait et théorique, est central en informatique ;  et on peut résoudre beaucoup de problèmes (semblant pourtant complétement différent) en les traduisant en des formules logiques.
On verra partie 2 l'exemple du Sudoku, mais bien d'autre (3-coloration, emplois du temps ...) pourraient être ainsi résolus <a name="cite_ref-2"></a>[<sup>[2]</sup>](#cite_note-2) de [problèmes](https://fr.wikipedia.org/wiki/Liste_de_probl%C3%A8mes_NP-complets) se réduisent (_ie_ peuvent être résolus grâce) à $\mathbf{SAT}$ (plus précisément à $\mathbf{3-SAT}$, qui [sert de référence](https://fr.wikipedia.org/wiki/Probl%C3%A8me_3-SAT) pour la [$\mathbf{NP}$-complétude](https://fr.wikipedia.org/wiki/Probl%C3%A8me_NP-complet))





<a name="cite_note-2"></a>2. [^](#cite_ref-2) les problèmes polynomiaux peuvent aussi être résolu par $\mathbf{SAT}$, ce n'est cependant pas pertinent pusiqu'ils sont justement polynomiaux tandis que $\mathbf{SAT}$ peut être exponentiel (sauf si $\mathbf{P} = \mathbf{NP})$

*********


En raison de cette importance, de nombreuses personnes cherchent à optimiser la résolution de formule, en créant des[SAT-Solver](https://en.wikipedia.org/wiki/SAT_solver) les plus efficaces possibles.

Dans ce TP, nous allons utiliser **pycosat**
https://pypi.org/project/pycosat/

La syntaxe de pycosat est légérement différente de la notre, comme le montre l'exemple suivant :


In [ ]:
pycosat.solve(cnf)

[1, -2, -3, -4, 5]

*********


# Sudoku


Le problème général (sur une grille $n\times n$) du sudoku est $\mathbf{NP}$-complet, on ne connait pas d'algorithme efficace pour le résoudre.
Une approche classique de résolution est le _backtracking_, nous allons de nôtre côté utiliser pycosat.

On se donne quelques grilles d'exemple (vous pouvez créer les vôtres).

In [ ]:



easy = [[0, 0, 0, 1, 0, 9, 4, 2, 7],
        [1, 0, 9, 8, 0, 0, 0, 0, 6],
        [0, 0, 7, 0, 5, 0, 1, 0, 8],
        [0, 5, 6, 0, 0, 0, 0, 8, 2],
        [0, 0, 0, 0, 2, 0, 0, 0, 0],
        [9, 4, 0, 0, 0, 0, 6, 1, 0],
        [7, 0, 4, 0, 6, 0, 9, 0, 0],
        [6, 0, 0, 0, 0, 8, 2, 0, 5],
        [2, 9, 5, 3, 0, 1, 0, 0, 0]]

medium = [[5, 8, 0, 0, 0, 1, 0, 0, 0],
        [0, 3, 0, 0, 6, 0, 0, 7, 0],
        [9, 0, 0, 3, 2, 0, 1, 0, 6],
        [0, 0, 0, 0, 0, 0, 0, 5, 0],
        [3, 0, 9, 0, 0, 0, 2, 0, 1],
        [0, 5, 0, 0, 0, 0, 0, 0, 0],
        [6, 0, 2, 0, 5, 7, 0, 0, 8],
        [0, 4, 0, 0, 8, 0, 0, 1, 0],
        [0, 0, 0, 1, 0, 0, 0, 6, 5]]

evil = [[0, 2, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 6, 0, 0, 0, 0, 3],
        [0, 7, 4, 0, 8, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 3, 0, 0, 2],
        [0, 8, 0, 0, 4, 0, 0, 1, 0],
        [6, 0, 0, 5, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 7, 8, 0],
        [5, 0, 0, 0, 0, 9, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 4, 0]]

hard = [[0, 2, 0, 0, 0, 0, 0, 3, 0],
        [0, 0, 0, 6, 0, 1, 0, 0, 0],
        [0, 6, 8, 2, 0, 0, 0, 0, 5],
        [0, 0, 9, 0, 0, 8, 3, 0, 0],
        [0, 4, 6, 0, 0, 0, 7, 5, 0],
        [0, 0, 1, 3, 0, 0, 4, 0, 0],
        [9, 0, 0, 0, 0, 7, 5, 1, 0],
        [0, 0, 0, 1, 0, 4, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0, 9, 0]]

blank = [[0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0]]
    




On va encoder une grille de la manière suivante : 
la variable $v_{(i,j,d)}$ représente "la cellule ligne $i$ colonne $j$ contient le chiffre $d$". Malheuresement c'est une représentation $3$d, et nos CNF sont $1$d, on va donc faire la transformation

$v_{(i,j,d)} \leadsto x_{81*(i-1) + 9*(j-1) + d}$


In [ ]:
 

    

#variable "the cell i,j contains digit d"
#example:  340 -> 81*(5-1) + 9*(2-1) + 7 means cell at row 5 and column 2 contains a 7

def v(i, j, d): 
    return 81 * (i - 1) + 9 * (j - 1) + d


def read_cell(sol,i, j):
    # return the digit of cell i, j according to the solution
    for d in range(1, 10):
        if v(i, j, d) in sol:
            return d

**_question 7_**

Écrire une fonction qui créé la CNF "sudoku" (sans contraintes)

In [ ]:
        
        
def clause(res,cells):
    #add clauses to res such that each cell in cells has a diff value
    for i, xi in enumerate(cells):
        for j, xj in enumerate(cells):
            if i < j:
                for d in range(1, 10):
                    res.append([-v(xi[0], xi[1], d), -v(xj[0], xj[1], d)])

        
    return res

In [ ]:
def v(i, j, d):
    #helper function to convert a (row, col, digit) tuple to a corresponding integer
    return (i-1)*81 + (j-1)*9 + (d-1) + 1


def sudoku_clauses(): 
    res = []
    
    # each cell contains at least one digit
    for i in range(1, 10):
        for j in range(1, 10):
            res.append([v(i, j, d) for d in range(1, 10)])

    # each digit appears at most once in each row
    for i in range(1, 10):
        for d in range(1, 10):
            for j1 in range(1, 9):
                for j2 in range(j1+1, 10):
                    res.append([-v(i, j1, d), -v(i, j2, d)])

    # each digit appears at most once in each column
    for j in range(1, 10):
        for d in range(1, 10):
            for i1 in range(1, 9):
                for i2 in range(i1+1, 10):
                    res.append([-v(i1, j, d), -v(i2, j, d)])

    # each digit appears at most once in each 3x3 block
    for bx in range(0, 3):
        for by in range(0, 3):
            for d in range(1, 10):
                for i_offset in range(0, 3):
                    for j_offset in range(0, 3):
                        i = 3*bx + i_offset + 1
                        j = 3*by + j_offset + 1
                        for i2_offset in range(0, 3):
                            for j2_offset in range(0, 3):
                                if i_offset == i2_offset and j_offset == j2_offset:
                                    continue
                                i2 = 3*bx + i2_offset + 1
                                j2 = 3*by + j2_offset + 1
                                res.append([-v(i, j, d), -v(i2, j2, d)])

    return res


def gridToCNF(grid=blank):
    #turns a grid into a CNF
    clauses = sudoku_clauses()
    
    # add clauses to enforce the given cells
    for i in range(1, 10):
        for j in range(1, 10):
            d = grid[i-1][j-1]
            if d != 0:
                clauses.append([v(i, j, d)])
    
    return clauses


def solve(grid=blank):

    # solve the SAT problem
    
    clauses =gridToCNF(grid)
    start = time.time()
    sol = set(pycosat.solve(clauses))
    end = time.time()
    print("Time: "+str(end - start))
    


    for i in range(1, 10):
        for j in range(1, 10):
            grid[i - 1][j - 1] = read_cell(sol,i, j)

            
            
def solve_problem(problemset):
    print('Problem:') 
    pprint(problemset)  
    solve(problemset) 
    print('Answer:')
    pprint(problemset) 

In [ ]:
solve_problem(hard)


Problem:
[[0, 2, 0, 0, 0, 0, 0, 3, 0],
 [0, 0, 0, 6, 0, 1, 0, 0, 0],
 [0, 6, 8, 2, 0, 0, 0, 0, 5],
 [0, 0, 9, 0, 0, 8, 3, 0, 0],
 [0, 4, 6, 0, 0, 0, 7, 5, 0],
 [0, 0, 1, 3, 0, 0, 4, 0, 0],
 [9, 0, 0, 0, 0, 7, 5, 1, 0],
 [0, 0, 0, 1, 0, 4, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 9, 0]]
Time: 1.7881393432617188e-05
Answer:
[[None, None, None, None, None, None, None, None, None],
 [None, None, None, None, None, None, None, None, None],
 [None, None, None, None, None, None, None, None, None],
 [None, None, None, None, None, None, None, None, None],
 [None, None, None, None, None, None, None, None, None],
 [None, None, None, None, None, None, None, None, None],
 [None, None, None, None, None, None, None, None, None],
 [None, None, None, None, None, None, None, None, None],
 [None, None, None, None, None, None, None, None, None]]


In [ ]:
pblm = gridToCNF(blank)

start = time.time()
sol = set(pycosat.solve(pblm))
end = time.time()
print("Time: "+str(end - start))

Time: 7.581710815429688e-05


In [ ]:
start = time.time()
sol = bruteSAT(pblm)
end = time.time()
print("Time: "+str(end - start))

ValueError: max() arg is an empty sequence

## Multiple solutions




In [ ]:
for sol in pycosat.itersolve(cnf):
    print(sol)

[1, -2, -3, -4, 5]
[1, -2, -3, 4, -5]
[1, -2, -3, 4, 5]
[1, -2, 3, -4, -5]
[1, -2, 3, -4, 5]
[1, 2, 3, -4, -5]
[1, 2, 3, -4, 5]
[1, 2, -3, -4, 5]
[1, 2, -3, 4, -5]
[1, 2, -3, 4, 5]
[-1, 2, -3, 4, -5]
[-1, 2, -3, 4, 5]
[-1, 2, -3, -4, -5]
[-1, 2, 3, -4, -5]
[-1, -2, 3, -4, -5]
[-1, -2, -3, -4, -5]
[-1, -2, -3, 4, 5]
[-1, -2, -3, 4, -5]


### Extensions suggérées :


#### Génération aléatoire de grilles (résolubles)

à l'aide des algorithmes précédents, écrire une fonction engendrant des grilles partielles de sudoku (résolubles)



#### 3-Coloriage

De nombreux problèmes importants en informatique (les problèmes inclus dans $\mathbf{NP}$ sont _réductibles_ à 3-SAT, c'est-à-dire qu'on peut les convertir en un problème 3-SAT et vis-et-versa (en temps polynomial, avec un nombre de d'inconnues polynomial). 

On peut donc (essayer de) résoudre de nombreux problème avec 3-SAT. Parmi eux la 3-coloration (étant donné un graphe $G$, admet-il une 3 coloration ?)

Écrire un programme qui, étant donné un graphe $G$ transforme 3-col en 3-SAT, résout le prolème à l'aide d'un SAT-solver, puis donne la solution en terme de graphe).


_note :_ Pour le sens inverse, vous pouvez vous référer à https://cgi.csc.liv.ac.uk/~igor/COMP309/3CP.pdf par exemple